In [2]:
import pandas as pd

In [12]:
xls = pd.ExcelFile('/work/Final_Data_PPE_v2_fz1.xlsx')

df1 = pd.read_excel(xls, 'Bridge_A')

In [16]:
# Current Ductility Demand is calculated by Rot100CDR / delta_y
df1['Collapse'] = (df1['Ductility Demand'] > 4).astype(int)
df1.head()

,Bridge_No,Record Sequence Number,5-95% Duration (sec),Earthquake Name,Year,Station Name,Magnitude,Mechanism,Rjb (km),Rrup (km),...,delta_y,Ry,Ductility,Period,Vy/W,Rot50CDR,Rot100CDR,Ductility Demand,DI,Collapse
0,1,18553,11.8,"""21273463""",2003,"""Bear Gulch Road""",3.3,Normal Oblique,51.36,51.84,...,0.009604,1.217737e-06,3.553846,1.079948,0.041191,0.000013,0.000016,0.001671,-0.390912,0
1,1,18554,13.6,"""21273463""",2003,"""Black Mountain Los Altos""",3.3,Normal Oblique,35.90,36.58,...,0.009604,6.290907e-07,3.553846,1.079948,0.041191,0.000006,0.000009,0.000910,-0.391210,0
2,1,18555,11.9,"""21273463""",2003,"""El Granada""",3.3,Normal Oblique,62.10,62.50,...,0.009604,4.044441e-07,3.553846,1.079948,0.041191,0.000004,0.000004,0.000442,-0.391393,0
3,1,18556,9.4,"""21273463""",2003,"""Huhtala Ln""",3.3,Normal Oblique,42.34,42.92,...,0.009604,2.017415e-06,3.553846,1.079948,0.041191,0.000013,0.000014,0.001455,-0.390997,0
4,1,18557,12.1,"""21273463""",2003,"""Saint Joseph's Hill""",3.3,Normal Oblique,29.76,30.58,...,0.009604,1.515664e-06,3.553846,1.079948,0.041191,0.000008,0.000011,0.001180,-0.391104,0


In [88]:
df1[' Earthquake Name'] = df1[' Earthquake Name'].str.replace('"', '')

# some earthquake names are strings. could we assign random int values to them??

In [39]:
df1.columns

Index(['Bridge_No', ' Record Sequence Number', ' 5-95% Duration (sec)',
       ' Earthquake Name', ' Year', ' Station Name', ' Magnitude',
       ' Mechanism', ' Rjb (km)', ' Rrup (km)', ' Vs30 (m/sec)', 'EQID',
       'Strike (deg)', 'Dip (deg)', 'Rake Angle (deg)', 'Mechanism',
       'Dhyp (km)', 'Ztor (km)', 'W (km)', 'L (km)', 'A (km)', 'HypD (km)',
       'Rx', 'FW/HW Indicator', 'Z2.5 (km)', 'A1100', 'Region', 'SJ',
       'delta_y', 'Ry', 'Ductility', 'Period', 'Vy/W', 'Rot50CDR', 'Rot100CDR',
       'Ductility Demand', 'DI', 'Collapse', 'Earthquake Name'],
      dtype='object')

In [47]:
nan_counts = df1.isna().sum()
print(nan_counts)

Bridge_No                      0
 Record Sequence Number        0
 5-95% Duration (sec)         30
 Earthquake Name               0
 Year                          0
 Station Name                  0
 Magnitude                     0
 Mechanism                     0
 Rjb (km)                      0
 Rrup (km)                     0
 Vs30 (m/sec)                  0
EQID                           0
Strike (deg)                   0
Dip (deg)                      0
Rake Angle (deg)               0
Mechanism                      0
Dhyp (km)                      0
Ztor (km)                      0
W (km)                         0
L (km)                         0
A (km)                         0
HypD (km)                      0
Rx                             0
FW/HW Indicator            19320
Z2.5 (km)                      0
A1100                        250
Region                         0
SJ                             0
delta_y                        0
Ry                             0
Ductility 

In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Doing this just to test
df1_cleaned = df1.dropna()

# Define the feature columns and target variable
feature_columns = [
       ' Earthquake Name', ' Year', ' Station Name', ' Magnitude',
       ' Mechanism', ' Rjb (km)', ' Rrup (km)', ' Vs30 (m/sec)', 'EQID',
       'Strike (deg)', 'Dip (deg)', 'Rake Angle (deg)', 'Mechanism',
       'Dhyp (km)', 'Ztor (km)', 'W (km)', 'L (km)', 'A (km)', 'HypD (km)',
       'Rx', 'Z2.5 (km)', 'Region', 'SJ',
       'Ry', 'Ductility', 'Period', 'Vy/W', 'Rot50CDR']
target_column = 'Collapse'

# Extract features and target variable
X = df1[feature_columns]
y = df1[target_column]

# One-hot encode categorical features
X_encoded = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get feature importances
feature_importance = model.feature_importances_

# Create a DataFrame to store feature names and their importances
feature_importance_df = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': feature_importance
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the top features
print(feature_importance_df.head())

Accuracy: 0.99469305794607
       Feature  Importance
24    Rot50CDR    0.238588
20          Ry    0.138021
3    Rrup (km)    0.064110
2     Rjb (km)    0.063884
23        Vy/W    0.043635


In [87]:
from sklearn.metrics import classification_report

# Detailed classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


# TODO: Fine tune to increase accuracy for collapse cases at expense of non-collapse accuracy

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     13756
           1       0.89      0.69      0.78       188

    accuracy                           0.99     13944
   macro avg       0.94      0.85      0.89     13944
weighted avg       0.99      0.99      0.99     13944



In [79]:
print(df1.shape)
print(df1_cleaned.shape)

(69720, 38)
(50400, 38)


In [80]:
df_collapsed = df1[df1['Collapse']==1]
df_collapsed

,Bridge_No,Record Sequence Number,5-95% Duration (sec),Earthquake Name,Year,Station Name,Magnitude,Mechanism,Rjb (km),Rrup (km),...,delta_y,Ry,Ductility,Period,Vy/W,Rot50CDR,Rot100CDR,Ductility Demand,DI,Collapse
1071,1,723,11.0,Superstition Hills-02,1987,"""Parachute Test Site""",6.54,strike slip,0.95,0.95,...,0.009604,0.016249,3.553846,1.079948,0.041191,0.131791,0.179705,18.711486,6.935220,1
1080,1,732,11.8,Loma Prieta,1989,"""APEEL 2 - Redwood City""",6.93,Reverse Oblique,43.06,43.23,...,0.009604,0.019233,3.553846,1.079948,0.041191,0.043442,0.055320,5.760096,1.863893,1
1089,1,741,9.8,Loma Prieta,1989,"""BRAN""",6.93,Reverse Oblique,3.85,10.72,...,0.009604,0.008702,3.553846,1.079948,0.041191,0.035426,0.046848,4.877937,1.518469,1
1100,1,753,7.9,Loma Prieta,1989,"""Corralitos""",6.93,Reverse Oblique,0.16,3.85,...,0.009604,0.008045,3.553846,1.079948,0.041191,0.037136,0.045593,4.747275,1.467306,1
1106,1,759,23.1,Loma Prieta,1989,"""Foster City - APEEL 1""",6.93,Reverse Oblique,43.77,43.94,...,0.009604,0.010247,3.553846,1.079948,0.041191,0.032429,0.045915,4.780842,1.480450,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65373,10,182,6.8,Imperial Valley-06,1979,"""El Centro Array #7""",6.53,strike slip,0.56,0.56,...,0.007153,0.004860,6.085000,1.137262,0.056941,0.027798,0.038104,5.326873,0.850909,1
65424,10,319,6.9,Westmorland,1981,"""Westmorland Fire Sta""",5.90,strike slip,6.18,6.50,...,0.007153,0.004578,6.085000,1.137262,0.056941,0.023466,0.041832,5.848173,0.953426,1
65441,10,338,13.2,Coalinga-01,1983,"""Parkfield - Fault Zone 14""",6.36,Reverse,28.11,29.48,...,0.007153,0.007277,6.085000,1.137262,0.056941,0.021330,0.032473,4.539788,0.696124,1
65476,10,451,4.1,Morgan Hill,1984,"""Coyote Lake Dam - Southwest Abutment""",6.19,strike slip,0.18,0.53,...,0.007153,0.005470,6.085000,1.137262,0.056941,0.028284,0.043618,6.097760,1.002509,1


In [74]:
print(df1['Collapse'].value_counts())

Collapse
0    68821
1      899
Name: count, dtype: int64


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd42330f-db8e-4ceb-ade1-2d5f297f211b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>